# HDD Predictive Maintenance - Felix - SQL Getting the Data

In this notebook we treat the prediction of HDD breakdown as a classification problem. We think about what data to use and pull it from the SQL server.

Since we have a hugely imbalanced dataset, we check what are the most frequent models in the dataset in our time period. Then we will focus on one model first and pick drive snapshots in the 30 days before they fail. Additionally, we pick several drive snapshots further before failing.

## Setup workspace and connect to SQL

In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from tqdm import tqdm

RSEED = 42

Make sure to store the connection details in a `.env` file.

In [2]:
# Load env file with connection details
load_dotenv("../../.env")
DB_STRING = os.getenv('DB_STRING')
# Create engine to use with pandas
db = create_engine(DB_STRING)

## Model selection

First, we query the failures by model.

In [264]:
query_string = """SELECT model, COUNT(model), failure FROM "2021"
                GROUP BY model, failure
                HAVING failure = '1'
                ORDER BY count DESC"""
count_failed_by_model = pd.read_sql(query_string, db)
#count_failed_by_model = pd.read_csv("../data/interim/count_failed_by_model.csv")
count_failed_by_model.head(5)

,model,count,failure
0,ST4000DM000,610,1
1,ST12000NM0007,494,1
2,ST8000NM0055,392,1
3,ST12000NM0008,366,1
4,TOSHIBA MG07ACA14TA,347,1


We focus on the model with the most recent failures in order to obtain the most data. In our case that's the Seagate ST4000DM000.

## Querying true observations

We pick the drive snapshots of the 250 most recent failures of our model on the day they failed (reported `failure == 1`). This dataframe is an index of the drives that we use in the following.

In [49]:
query_string = """SELECT * FROM "2021"
                WHERE model = 'ST4000DM000' AND failure = '1'
                ORDER BY date DESC
                LIMIT 250"""
df_failed = pd.read_sql(query_string, db, parse_dates="date")
df_failed.head(5)

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2021-12-31,Z304GSQ0,ST4000DM000,4000787030016,1,104,12689626,None,None,91,...,None,None,None,None,None,None,None,None,None,None
1,2021-12-30,Z305AQQP,ST4000DM000,4000787030016,1,117,144131312,None,None,91,...,None,None,None,None,None,None,None,None,None,None
2,2021-12-28,Z30149L0,ST4000DM000,4000787030016,1,114,72330272,None,None,92,...,None,None,None,None,None,None,None,None,None,None
3,2021-12-26,Z302SWBN,ST4000DM000,4000787030016,1,108,16925000,None,None,91,...,None,None,None,None,None,None,None,None,None,None
4,2021-12-24,Z305K296,ST4000DM000,4000787030016,1,119,218225696,None,None,93,...,None,None,None,None,None,None,None,None,None,None


We use those HDDs and pick a snapshot of at a random date between 0 and 30 days before they fail. Those observations will obtain the label `fail_30_days == True` indicating that they will fail in the next 30 days.

In [50]:
df = df_failed.copy() # Copy dataframe
failed_numbers = df.iloc[:,:2]
# Loop over all the HDDs in our dataframe
for num, date, serial_number in tqdm(failed_numbers.itertuples()):
    date = date - pd.Timedelta(np.random.randint(1,30), 'days') # Substract random number of days
    # Generate SQL query
    query_string = f"""SELECT * FROM "2021" WHERE date = '{date.strftime('%Y-%m-%d')}' AND serial_number = '{serial_number}'"""
    snapshot = pd.read_sql(query_string, db, parse_dates="date")
    #snapshot = df.iloc[num,:]
    df = df.append(snapshot, ignore_index=True) # Append the snapshot to the dataframe
df = df.infer_objects() # Fix column types
df.shape

250it [00:30,  8.22it/s]


(485, 179)

Set the target:

In [51]:
df["fail_30_days"] = 1

## Querying false observations

Additionally, we capture 5 snapshots at random times between 600 and 30 days before they break down.

In [53]:
for k in range(5): # Repeat 5 times
    # Loop over all the HDDs in our dataframe
    for num, date, serial_number in tqdm(failed_numbers.itertuples()):
        date = date - pd.Timedelta(np.random.randint(31,600), 'days') # Substract random number of days
        # Generate SQL query
        query_string = f"""SELECT * FROM "2021" WHERE date = '{date.strftime('%Y-%m-%d')}' AND serial_number = '{serial_number}'"""
        snapshot = pd.read_sql(query_string, db, parse_dates="date")
        #snapshot = df.iloc[num,:]
        df = df.append(snapshot, ignore_index=True) # Append the snapshot to the dataframe
#df = pd.read_csv("../data/interim/felix.csv", parse_dates=["date"])
df = df.infer_objects() # Fix column types
df.shape

250it [00:31,  7.91it/s]
250it [00:31,  7.92it/s]
250it [00:31,  7.89it/s]
250it [00:31,  7.93it/s]
250it [00:31,  7.85it/s]


Setting the target

In [55]:
df["fail_30_days"][486:] = 0

Now we drop the first 250 rows, corresponding to the snapshots at the failure day.

In [70]:
#df.to_csv("../data/interim/felix.csv", index=False)
df.drop(df.index[:250], axis=0, inplace=True) # Drop first 250 rows
df.drop("failure", axis=1, inplace=True) # Drop the old target
df.reset_index(drop=True, inplace=True) # Reset index

Instead of pulling data at random dates, we could do random sampling in SQL directly:

In [7]:
## Random sampling in SQL
# SELECT * FROM myTable
# WHERE attribute = 'myValue'
# ORDER BY random()
# LIMIT 1000;

## Saving the data

In [81]:
df_backup = df.copy()
df.to_csv("../data/interim/felix.csv", index=False)